# Testing - MCP
## Introduction

## Installation

Note : we are pinning the inspect_ai due to a recent breaking change

In [2]:
%pip install -q openai anthropic ipywidgets colorama mcp
import os
os.environ['XDG_RUNTIME_DIR']="/tmp"
os.environ['INSPECT_EVAL_MODEL'] = "openai/gpt-4o-mini"

from helpers.reporter.pretty import pretty_results


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Adding MCP stdio

# these ar installed in the docker image
#%pip install -q mcp mcp-server-tree-sitter mcp-server-git

In [3]:
from inspect_ai.tool import mcp_server_sandbox, mcp_server_stdio
from inspect_ai import Task, eval, task

from inspect_ai.scorer import includes
from inspect_ai.dataset import Sample
from inspect_ai.agent import react

@task
def basic_mcp():
    patrick_scanner = mcp_server_stdio(
        command="python3",
        args=["./tools/patrick_scanner.py"]
    )

    patrick_fixer = mcp_server_stdio(
        command="python3",
        args=["./tools/patrick_fixer.py"]
    )

    git_server = mcp_server_stdio(
        command="python3",
        args=["-m", "mcp_server_git", "--repository", "/my-repo"]
    )

    treesitter_server = mcp_server_stdio(
            command="python3",
            args=["-m","mcp_server_tree_sitter.server" ]
        )

    treesitter_server_sandbox = mcp_server_sandbox(
         command="python3",
            args=["-m","mcp_server_tree_sitter.server" ],
      #      sandbox="mcp"
    )

    fs_server = mcp_server_sandbox(
       # sandbox="mcp",
        command="mcp-server-filesystem",
        args=["/"]
    )

    return Task(
        dataset=[Sample(
            "What can you tell me about the current project in /my-repo? And check for syntax errors",
        )],
        sandbox=("docker","compose.yaml"), # yaml contains the default for a sandbox
        solver=[react(tools=[treesitter_server_sandbox, fs_server])],
        
        scorer=[
        ],

        # solver=[use_tools=patrick_fixer])
    )

results = eval(basic_mcp,log_level="info",display="none",ftime_limit=60)
print(pretty_results(results))

Compose can now delegate builds to bake for better performance.
 To do so, set COMPOSE_BAKE=true.
#0 building with "default" instance using docker driver

#1 [mcp internal] load build definition from Dockerfile.mcp
#1 transferring dockerfile: 1.64kB done
#1 DONE 0.0s

#2 [mcp internal] load metadata for docker.io/aisiuk/inspect-tool-support:latest
#2 DONE 0.0s

#3 [mcp internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [mcp  1/11] FROM docker.io/aisiuk/inspect-tool-support:latest
#4 DONE 0.0s

#5 [mcp  7/11] RUN npm install -g @anthropic-ai/claude-code
#5 CACHED

#6 [mcp  3/11] RUN pip install aider-chat
#6 CACHED

#7 [mcp  5/11] RUN python3 -m playwright install --with-deps chromium
#7 CACHED

#8 [mcp  4/11] RUN pip install playwright
#8 CACHED

#9 [mcp  6/11] RUN npm install -g @modelcontextprotocol/server-filesystem
#9 CACHED

#10 [mcp 10/11] RUN pip install mcp-server-git
#10 CACHED

#11 [mcp  8/11] RUN npm install -g @openai/codex@0.1.2504211509
#11 CAC

Output()

 mcp  Built


Status: success Model: openai/gpt-4o-mini
/workspaces/workshop-testing/lessons/04-testing/logs/2025-05-07T13-07-27+00-00_basic-mcp_eb2nX5Qmn2dM84eAAwz9iP.eval
======= Sample ========
input : What can you tell me about the current project in /my-repo? And check for syntax errors
target: 
 system     > 
You are a helpful assistant attempting to submit the best possible answer.
You have several tools available to help with finding the answer. You will
see the result of tool calls right after sending the message. If you need
to perform multiple actions, you can always send more messages with additional
tool calls. Do some reasoning before your actions, describing what tool calls
you are going to use and how they fit into your plan.

When you have completed the task and have an answer, call the submit()
tool to report it.

 user       > What can you tell me about the current project in /my-repo? And check for syntax errors
 assistant [tool:analyze_project] > {'project': 'my-repo', 'scan_dep